In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import zipfile


zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/MURA-v1.1.zip", 'r')
zip_ref.extractall("/tmp/SML")
zip_ref.close()

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import cv2
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import os
from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
np.random.seed(1)

In [0]:
df = pd.read_csv("/tmp/SML/MURA-v1.1/train_image_paths.csv",
                 names=["x_col"])

df['Region'] = df["x_col"].map(lambda x: x.split('/')[-4])
df["Region"] = df["Region"].map(lambda x: x.split('XR_')[-1])
df["y_col"] = df["x_col"].map(lambda x: x.split('/')[-2])
df["y_col"] = df["y_col"].map(lambda x: x.split('_')[-1])
df


In [0]:
df_test = pd.read_csv("/tmp/SML/MURA-v1.1/valid_image_paths.csv",
                 names=["x_col"])
df_test['Region'] = df_test["x_col"].map(lambda x: x.split('/')[-4])
df_test["Region"] = df_test["Region"].map(lambda x: x.split('XR_')[-1])
df_test["y_col"] = df_test["x_col"].map(lambda x: x.split('/')[-2])
df_test["y_col"] = df_test["y_col"].map(lambda x: x.split('_')[-1])
df_test

In [0]:
X = df["x_col"]
y = df["Region"]

In [0]:
X_train_path, X_test_path, y_train, y_test = train_test_split(X, y, test_size = 0.20,stratify=y, random_state = 57)

In [0]:
df_train=pd.DataFrame()
df_valid = pd.DataFrame()
df_train["x_col"]=X_train_path
df_train["y_col"]=y_train
df_valid["x_col"]=X_test_path
df_valid["y_col"]=y_test

In [0]:
from keras.applications.resnet import ResNet152, preprocess_input
from keras.optimizers import Adam
from keras.models import Model
# DATASET_PATH  = '/tmp/SML/MURA-v1.1/train/WRIST'
IMAGE_SIZE    = (224, 224)
NUM_CLASSES   = 7
BATCH_SIZE    = 16  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 2  # freeze the first this many layers for training
NUM_EPOCHS    = 15
WEIGHTS_FINAL = 'model-resnet152-final.h5'


train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True)

train_batches = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory='/tmp/SML',
        x_col="x_col",
        y_col="y_col",
        interpolation='bicubic',
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True,
        seed =57)


# train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
#                                                   target_size=IMAGE_SIZE,
#                                                   interpolation='bicubic',
#                                                   class_mode='binary',
#                                                   shuffle=True,
#                                                   batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_batches = valid_datagen.flow_from_dataframe(
        dataframe=df_valid,
        directory='/tmp/SML',
        x_col="x_col",
        y_col="y_col",
        target_size=IMAGE_SIZE,
        interpolation='bicubic',
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False,
        seed = 57)

# valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
#                                                   target_size=IMAGE_SIZE,
#                                                   interpolation='bicubic',
#                                                   class_mode='binary',
#                                                   shuffle=False,
#                                                   batch_size=BATCH_SIZE)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_batches=test_datagen.flow_from_dataframe(
dataframe=df_test,
directory="/tmp/SML",
x_col="x_col",
y_col=None,
batch_size=1,
seed=57,
shuffle=False,
class_mode=None,
target_size=IMAGE_SIZE)

# show class indices
print('****************')
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))
print('****************')

# build our classifier model based on pre-trained ResNet152:
# 1. we don't include the top (fully connected) layers of ResNet152
# 2. we add a DropOut layer followed by a Dense (fully connected)
#    layer which generates softmax class score for each class
# 3. we compile the final model using an Adam optimizer, with a
#    low learning rate (since we are 'fine-tuning')
net = ResNet152(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(7, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
# print(net_final.summary())

# train the model
history = net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# save trained weights
net_final.save(WEIGHTS_FINAL)

In [0]:
test_batches.reset()
pred=net_final.predict_generator(test_batches,
steps=test_batches.samples//1,
verbose=1)

In [0]:
predicted_class_indices=np.argmax(pred,axis=1)

In [0]:
labels = (train_batches.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
df_test["pred"]=predictions
y_test = df_test["Region"]
y_pred = df_test["pred"]
labelencoder_y = LabelEncoder()
y_test = labelencoder_y.fit_transform(y_test)
y_pred = labelencoder_y.fit_transform(y_pred)

In [0]:
from sklearn import metrics
print("Accuracy on testing set:",metrics.accuracy_score(y_test, y_pred))
print("Error on testing set:",1-metrics.accuracy_score(y_test, y_pred))
print("\n")

In [0]:
from mlxtend.evaluate import confusion_matrix
cm = confusion_matrix(y_target=y_test, 
                      y_predicted=y_pred, 
                      binary=False)

In [0]:

recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)
print("Precision for ELBOW:", precision[0])
print("Precision for FINGER:", precision[1])
print("Precision for FOREARM:", precision[2])
print("Precision for HAND:", precision[3])
print("Precision for HUMERUS:", precision[4])
print("Precision for SHOULDER:", precision[5])
print("Precision for WRIST:", precision[6],"\n")
print("Recall for ELBOW:", recall[0])
print("Recall for FINGER:", recall[1])
print("Recall for FOREARM:", recall[2])
print("Recall for HAND:", recall[3])
print("Recall for HUMERUS:", recall[4])
print("Recall for SHOULDER:", recall[5])
print("Recall for WRIST:", recall[6],"\n")
print("Overall Recall",np.mean(recall))
print("Overall Precision", np.mean(precision))


In [0]:
import matplotlib.pyplot as plt
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=cm)

plt.show()

In [0]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()